In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/MyDrive/Capstone/

Mounted at /content/gdrive


/content/gdrive/MyDrive/Capstone


In [ ]:
!pip install pytorch_pretrained_bert pytorch-nlp

     |████████████████████████████████| 133kB 6.5MB/s 
     |████████████████████████████████| 92kB 5.0MB/s 
     |████████████████████████████████| 133kB 7.8MB/s 
     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 7.4MB 8.7MB/s 
ERROR: botocore 1.20.59 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.


In [ ]:
!pip install torchtext

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 6.3MB/s 
     |████████████████████████████████| 901kB 21.6MB/s 
     |████████████████████████████████| 3.3MB 29.2MB/s 


## Data Preparation

In [ ]:
# data analysis libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# data preprocessing libraries
import re
from datetime import datetime
import spacy
import nltk
from nltk import word_tokenize, sent_tokenize
from spacy_preprocessor import SpacyPreprocessor
from sklearn.model_selection import train_test_split

# other
from tqdm import tqdm, tqdm_notebook, tqdm_pandas

tqdm.pandas()
tqdm_notebook().pandas()

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import random

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
len(tokenizer.vocab)

30522

In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token, eos_token, pad_token, unk_token)
print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

[CLS] [SEP] [PAD] [UNK]
101 102 0 100


In [ ]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
print(max_input_length)

512


In [ ]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [ ]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [ ]:
fields = {'text': ('text',TEXT), 'target': ('label', LABEL)}

In [ ]:
sui_df = pd.read_csv('data/sui_df.csv')

In [ ]:
sui_df.head()

,text,target
0,"I just want to die Seriously, what’s the fuck ...",1
1,Suicidal thoughts have now turned into plans i...,1
2,My friend passed away a couple of years ago re...,1
3,I hid a noose in my backpack and I take it eve...,1
4,Fck this Pandemic I’ve been suicidal for many ...,1


In [ ]:
train_data, valid_data, test_data = data.TabularDataset.splits(
    path = 'data', 
    train = 'sui_df_train.csv',
    validation = 'sui_df_val.csv',
    test = 'sui_df_test.csv',
    format ='csv',
    fields = fields,
    skip_header = False
)

In [ ]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 7276
Number of validation examples: 1285
Number of testing examples: 2141


In [ ]:
LABEL.build_vocab(train_data)

In [ ]:
BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_within_batch = True,
    sort_key = lambda x: len(x.text),
    batch_size = BATCH_SIZE, 
    device = device)

In [ ]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
import torch.nn as nn

class BERT_Classifier(nn.Module):
    def __init__(self, bert, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        
        super().__init__()
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional = bidirectional, batch_first = True, dropout = 0 if n_layers < 2 else dropout)
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
          
        with torch.no_grad():
            embedded = self.bert(text)[0]
        _, hidden = self.rnn(embedded)
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        
        output = self.out(hidden)
        
        return output, hidden

In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERT_Classifier(bert, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

# Gotta freeze all BERT layers. Not going to retrain those.
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters
The model has 2,759,169 trainable parameters


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

print(optimizer, criterion, model)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    weight_decay: 0
) BCEWithLogitsLoss() BERT_Classifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text)[0].squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text)[0].squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'SUIBERT_RNN.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 48s
	Train Loss: 0.231 | Train Acc: 90.33%
	 Val. Loss: 0.157 |  Val. Acc: 95.12%
Epoch: 02 | Epoch Time: 0m 47s
	Train Loss: 0.136 | Train Acc: 94.87%
	 Val. Loss: 0.124 |  Val. Acc: 95.58%
Epoch: 03 | Epoch Time: 0m 47s
	Train Loss: 0.101 | Train Acc: 96.21%
	 Val. Loss: 0.194 |  Val. Acc: 93.22%
Epoch: 04 | Epoch Time: 0m 48s
	Train Loss: 0.080 | Train Acc: 97.09%
	 Val. Loss: 0.154 |  Val. Acc: 94.82%
Epoch: 05 | Epoch Time: 0m 47s
	Train Loss: 0.054 | Train Acc: 98.00%
	 Val. Loss: 0.152 |  Val. Acc: 94.97%


In [ ]:
model.load_state_dict(torch.load('SUIBERT_RNN.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.115 | Test Acc: 95.98%


In [ ]:
def predict_suicidality(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor)[0])
    return prediction.item(), model(tensor)[1]

In [ ]:
predict_suicidality(model, tokenizer, "I want to kill myself")[0]

0.9898316264152527

In [ ]:
predict_suicidality(model, tokenizer, "I have been feeling like I want to die")[0]

0.9817752838134766

In [ ]:
predict_suicidality(model, tokenizer, "I have been feeling depressed")[0]

0.6339775919914246

In [ ]:
predict_suicidality(model, tokenizer, "I have been feeling anxious. Things don't make sense. I cut myself")[0]

0.9718927145004272

In [ ]:
len(predict_suicidality(model, tokenizer, "lol what haha")[1][0])

512

In [ ]:
a = predict_suicidality(model, tokenizer, "I have been cutting myself")[1].detach().cpu().numpy().flatten()
b = predict_suicidality(model, tokenizer, "I want to harm myself")[1].detach().cpu().numpy().flatten()

In [ ]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a,b):
  cossim = dot(a, b)/(norm(a)*norm(b))
  return cossim

In [ ]:
cos_sim(a,b)

0.082763664

In [ ]:
total_df = pd.read_csv('data/total_df.csv', low_memory=False)

In [ ]:
total_df.head(3)

,username,type,subreddit,date,text,ups,downs,num_response,target,first_sw_post,word_count,sent_count,last_active_date,first_active_date,dtl,num_activity
0,BIGBOIBICH,0,depression,2021-04-01 09:16:14,Honestly people don’t know about the fact that...,16,0,0,0,NaN,62,2,2021-04-02 14:23:52,2019-01-24 08:01:01,1.0,138
1,BIGBOIBICH,0,depression,2021-03-31 21:46:48,Ye I’ll try to today thank you for the tip,1,0,0,0,NaN,10,1,2021-04-02 14:23:52,2019-01-24 08:01:01,1.0,138
2,BIGBOIBICH,0,depression,2021-03-31 20:43:14,I’m sorry to hear that I really am but I hope ...,1,0,0,0,NaN,14,1,2021-04-02 14:23:52,2019-01-24 08:01:01,1.0,138


In [ ]:
def get_tokens(text):
  tokens = tokenizer.tokenize(text)
  tokens = tokens[:max_input_length-2]
  indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
  return indexed

In [ ]:
indx = total_df['text'].progress_apply(get_tokens)

In [ ]:
#tensor = torch.LongTensor(indx.to_list())
tensor = list(map(lambda x: torch.LongTensor(x).unsqueeze(0).to(device), indx.to_list()))

In [ ]:
model.eval()
list(map(lambda x: model(x)[1][0], tensor))

[tensor([0.4290, 0.5071, 0.4846, 0.5248, 0.5246, 0.5528, 0.4257, 0.3480, 0.3682,
         0.5316, 0.4641, 0.6370, 0.5375, 0.5522, 0.3557, 0.5734, 0.3289, 0.6620,
         0.4387, 0.3698, 0.5594, 0.5133, 0.3353, 0.3935, 0.4382, 0.4151, 0.4935,
         0.5326, 0.5320, 0.5717, 0.6651, 0.5594, 0.3842, 0.5464, 0.3572, 0.4947,
         0.3072, 0.4850, 0.3552, 0.4737, 0.5620, 0.5774, 0.5712, 0.4419, 0.4003,
         0.5351, 0.5434, 0.4419, 0.4754, 0.5219, 0.6213, 0.5380, 0.4684, 0.5987,
         0.5744, 0.5121, 0.3147, 0.5030, 0.5877, 0.6267, 0.5087, 0.5974, 0.3922,
         0.6468, 0.5114, 0.5330, 0.5080, 0.6282, 0.6467, 0.2950, 0.4868, 0.4794,
         0.4077, 0.4914, 0.3957, 0.4687, 0.5089, 0.5165, 0.4980, 0.4098, 0.6253,
         0.5168, 0.4537, 0.5997, 0.4481, 0.6050, 0.4434, 0.4194, 0.4346, 0.5952,
         0.5875, 0.6000, 0.5617, 0.5074, 0.6073, 0.4021, 0.4667, 0.5716, 0.4398,
         0.3811, 0.3723, 0.6675, 0.3353, 0.5035, 0.4185, 0.6503, 0.3864, 0.6444,
         0.4253, 0.4541, 0.5

In [ ]:
  tensor = torch.LongTensor(indexed).to(device)
  tensor = tensor.unsqueeze(0)
  prediction = torch.sigmoid(model(tensor)[0])

In [ ]:
total_df['activity_id'] 

In [ ]:
fields = {'text': ('text',TEXT), 'target': ('label', LABEL)}

KeyboardInterrupt: ignored

In [ ]:
tokenizer.tokenize(["I have been feeling like I want to die","lol what haha"])

TypeError: ignored